# Quality Analysis over the DB search with the models

In [1]:
import boto3
import csv
import json
import random

## 

In [2]:
# path = 'C:/Users/jordi/Documents/claus/'
path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [10]:
language = "spanish"

In [11]:
# The code below is to pick random document codes from the list of documents with incentives
model = "simple_transformers"
path = "../output/"
# file_name = f"incentives_{language}.csv"
file_name = f"incentives_{model}_{language}.csv"
file = path + file_name

with open(file, newline='') as f:
    reader = csv.reader(f)
    next(reader)
    file_code = [row[0] for row in reader]

codes = {}
for item in random.sample(file_code, 50):
    codes[item] = 0

In [12]:
# To pick the sentences refering to incentives from the selected documents

# in_prefix = f"{language}_documents/HSSC/updated_sentences/"  
in_prefix = f"{language}_documents/HSSC/{model}/updated_sentences/"  

i = 0
documents_with_incentives = 0
results_list = [["Sentence_name", "Sentence text", "Label"]]

incentives = 0
for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):
    name = obj.key.split("/")[-1].split("_")[0]
#     print(name)
    if name in codes :
        sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
        for sentence_id in sentences[name]["sentences"]:
            incentive = sentences[name]["sentences"][sentence_id]["label"]
            if len(incentive) > 1 and incentive[0] == "Incentive":
                incentives += 1
                results_list.append([])
                results_list[incentives].append(name)
                results_list[incentives].append(sentences[name]["sentences"][sentence_id]["text"])
                results_list[incentives].append(incentive[1])
                
with open(f"../output/incentive_sentences_{model}_{language}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(results_list)           

In [13]:
# The code below is to retrieve de document codes of the sentences which refer to incentives and 
# that has been already picked for QA

path = "../output/"
file_name = f"incentive_sentences_{model}_{language}.csv"
file = path + file_name

with open(file, newline='') as f:
    reader = csv.reader(f)
    next(reader)
    file_code = [row[0].split("_")[0] for row in reader]

codes = {}
for item in file_code:
    codes[item] = 0

In [14]:
# To pick up the sentences refering to non-incentives from the selected documents

in_prefix = f"{language}_documents/HSSC/{model}/updated_sentences/"  

i = 0
documents_with_incentives = 0
results_list = [["Sentence_name", "Sentence text", "Label"]]


for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix):
    name = obj.key.split("/")[-1].split("_")[0]
#     print(name)
    if name in codes :
        sentences = json.loads(obj.get()['Body'].read().decode('utf-8'))
        n_incentives = incentive = sentences[name]["metadata"]["number_of_incentives"]
        print(n_incentives)
        non_incentives = [[]]
        incentives = 0
        for sentence_id in sentences[name]["sentences"]:
            incentive = sentences[name]["sentences"][sentence_id]["label"]
            if len(incentive) == 1 and incentive[0] == "not_Incentive":
                non_incentives.append([])
                incentives += 1
                non_incentives[incentives].append(name)
                non_incentives[incentives].append(sentences[name]["sentences"][sentence_id]["text"])
                non_incentives[incentives].append(incentive[0])
        
        for item in random.sample(non_incentives, n_incentives):
            results_list.append(item)
                
                
with open(f"../output/non_incentive_sentences_{model}_{language}.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(results_list)          

1
1
1
13
4
2
5
10
9
1
2
13
11
1
1
1
4
2
1
1
8
4
12
18
1
5
4
793
1
13
8
1
1
2
1
1
8
244
8
1
5
5
1
1
1
1
6
6
13
2
